In [1]:
import pandas as pd
import numpy as np
import datetime

data = pd.read_excel('last.xlsx', sheet_name='ГРП')
dummy = pd.read_excel('last.xlsx', sheet_name='МЭР')
dummy

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт
0,1974-11-01,1003,139.51,116.35,0.00,NaN,NaN,Ю1
1,1974-12-01,1003,39.06,32.58,0.00,NaN,NaN,Ю1
2,1975-01-01,1003,27.90,23.27,0.00,NaN,NaN,Ю1
3,1975-02-01,1003,27.86,23.24,0.00,NaN,NaN,Ю1
4,1975-03-01,1003,20.93,17.45,0.00,NaN,NaN,Ю1
...,...,...,...,...,...,...,...,...
59439,2005-09-01,5247,1.99,0.23,85.91,216.0,176.1,Ю1
59440,2005-10-01,5247,5.70,0.58,87.73,216.0,126.2,Ю1
59441,2005-11-01,5247,7.76,1.00,84.55,216.0,100.6,Ю1
59442,2005-12-01,5247,5.65,0.71,84.93,216.0,99.2,Ю1


In [2]:
#date_column_name = 'Дата ВНР после ГС \\ ГРП \\ЗБГС'
target = 'Скважина №'
data['Скважина №'] = data.apply(
            lambda x:
            str(x['Скважина №']), axis=1)
dummy['Скважина №'] = dummy.apply(
            lambda x:
            str(x['Скважина №']), axis=1)

In [3]:
data['для_грп'] = data.apply(
                lambda x:
                x['Скважина №'].partition('_')[0] if '_' in str(x['Скважина №'])
                else x['Скважина №'], axis=1)
no_grp = data.groupby('для_грп').agg(
                {'ГС/ННС': lambda x:
                x.tolist()}
            )
no_grp.reset_index(inplace=True)
no_grp['ГРП?'] = no_grp.apply(
                lambda x:
                0 if 'ГРП' not in str(x['ГС/ННС'])
                else 1, axis=1)
no_grp = no_grp.drop(no_grp[no_grp['ГРП?'] == 1].index)
spisok = no_grp['для_грп'].tolist()
no_grp

,для_грп,ГС/ННС,ГРП?
33,1552,[ГС],0
77,822,[ГС],0


In [4]:

step = data.groupby('для_грп').agg(
                {'Дата ВНР после ГС \\ ГРП \\ЗБГС': lambda x:
                x.tolist()[0]
                if len(x.tolist()) == 1 else x.tolist()[1]}
            )


KeyError: "Column(s) ['Дата ВНР после ГС \\\\ ГРП \\\\ЗБГС'] do not exist"

In [ ]:
step.reset_index(inplace=True)
step = step[~step['для_грп'].isin(spisok)].reset_index(drop=True)
vvv = step['для_грп'].tolist()

step['Скважина №'] = vvv
step = step.drop(['для_грп'], axis=1)
dummy = dummy.merge(step, on=target)
dummy[target] = dummy.apply(
                lambda x:
                str(x[target])
                + ('_ГРП' if x['Дата'] > x['Дата ВНР после ГС \\ ГРП \\ЗБГС'] else ''),
                axis=1
            )
dummy = dummy.drop('Дата ВНР после ГС \\ ГРП \\ЗБГС', axis=1)


In [ ]:
dummy